# Chapter 5 : Retrieving, Processing, and Storing Data

In [1]:
#load watermark
%load_ext watermark
%watermark -a 'Gopala KR' -u -d -v -p watermark,numpy,pandas,matplotlib,nltk,sklearn,tensorflow,theano,mxnet,chainer,seaborn,keras,tflearn,bokeh,gensim

/srv/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Using TensorFlow backend.
/srv/venv/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


Gopala KR 
last updated: 2018-02-20 

CPython 3.6.3
IPython 6.2.1

watermark 1.6.0
numpy 1.14.0
pandas 0.22.0
matplotlib 2.1.2
nltk 3.2.5
sklearn 0.19.1
tensorflow 1.5.0
theano 1.0.1
mxnet 1.1.0
chainer 3.4.0
seaborn 0.8.1
keras 2.1.4
tflearn n
bokeh 0.12.14
gensim 3.3.0


/srv/venv/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# Writing CSV files

In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)

a = np.random.randn(3, 4)
a[2][2] = np.nan
print(a)
np.savetxt('np.csv', a, fmt='%.2f', delimiter=',', header=" #1, #2,  #3,  #4")
df = pd.DataFrame(a)
print(df)
df.to_csv('pd.csv', float_format='%.2f', na_rep="NAN!")

[[ 0.49671415 -0.1382643   0.64768854  1.52302986]
 [-0.23415337 -0.23413696  1.57921282  0.76743473]
 [-0.46947439  0.54256004         nan -0.46572975]]
          0         1         2         3
0  0.496714 -0.138264  0.647689  1.523030
1 -0.234153 -0.234137  1.579213  0.767435
2 -0.469474  0.542560       NaN -0.465730


# Comparing binary .npy format and pickle format

In [3]:
import numpy as np
import pandas as pd
from tempfile import NamedTemporaryFile
from os.path import getsize

np.random.seed(42)
a = np.random.randn(365, 4)

tmpf = NamedTemporaryFile()
np.savetxt(tmpf, a, delimiter=',')
print("Size CSV file", getsize(tmpf.name))

tmpf = NamedTemporaryFile()
np.save(tmpf, a)
tmpf.seek(0)
loaded = np.load(tmpf)
print("Shape", loaded.shape)
print("Size .npy file", getsize(tmpf.name))

df = pd.DataFrame(a)
df.to_pickle(tmpf.name)
print("Size pickled dataframe", getsize(tmpf.name))
print("DF from pickle\n", pd.read_pickle(tmpf.name))


Size CSV file 36693
Shape (365, 4)
Size .npy file 11808
Size pickled dataframe 12254
DF from pickle
             0         1         2         3
0    0.496714 -0.138264  0.647689  1.523030
1   -0.234153 -0.234137  1.579213  0.767435
2   -0.469474  0.542560 -0.463418 -0.465730
3    0.241962 -1.913280 -1.724918 -0.562288
4   -1.012831  0.314247 -0.908024 -1.412304
5    1.465649 -0.225776  0.067528 -1.424748
6   -0.544383  0.110923 -1.150994  0.375698
7   -0.600639 -0.291694 -0.601707  1.852278
8   -0.013497 -1.057711  0.822545 -1.220844
9    0.208864 -1.959670 -1.328186  0.196861
10   0.738467  0.171368 -0.115648 -0.301104
11  -1.478522 -0.719844 -0.460639  1.057122
12   0.343618 -1.763040  0.324084 -0.385082
13  -0.676922  0.611676  1.031000  0.931280
14  -0.839218 -0.309212  0.331263  0.975545
15  -0.479174 -0.185659 -1.106335 -1.196207
16   0.812526  1.356240 -0.072010  1.003533
17   0.361636 -0.645120  0.361396  1.538037
18  -0.035826  1.564644 -2.619745  0.821903
19   0.087047 -0.29

In [5]:
!pip install tables

    100% |████████████████████████████████| 4.6MB 148kB/s eta 0:00:01
    100% |████████████████████████████████| 399kB 3.2MB/s eta 0:00:01


In [6]:
import numpy as np
import tables
from tempfile import NamedTemporaryFile
from os.path import getsize

np.random.seed(42)
a = np.random.randn(365, 4)

tmpf = NamedTemporaryFile()
h5file = tables.open_file(tmpf.name, mode='w', title="NumPy Array")
root = h5file.root
h5file.create_array(root, "array", a)
h5file.close()

h5file = tables.open_file(tmpf.name, "r")
print(getsize(tmpf.name))

for node in h5file.root:
   b = node.read()
   print(type(b), b.shape)

h5file.close()

13824
<class 'numpy.ndarray'> (365, 4)


# Reading and writing DataFrames to HDF5

In [7]:
import numpy as np
import pandas as pd
from tempfile import NamedTemporaryFile

np.random.seed(42)
a = np.random.randn(365, 4)

tmpf = NamedTemporaryFile()
store = pd.io.pytables.HDFStore(tmpf.name)
print(store)

df = pd.DataFrame(a)
store['df'] = df
print(store)

print("Get", store.get('df').shape)
print("Lookup", store['df'].shape)
print( "Dotted", store.df.shape)

del store['df']
print("After del\n", store)

print("Before close", store.is_open)
store.close()
print("After close", store.is_open)

df.to_hdf('test.h5', 'data', format='table')
print(pd.read_hdf('test.h5', 'data', where=['index>363']))

<class 'pandas.io.pytables.HDFStore'>
File path: /tmp/tmp83t_wzzg

<class 'pandas.io.pytables.HDFStore'>
File path: /tmp/tmp83t_wzzg

Get (365, 4)
Lookup (365, 4)
Dotted (365, 4)
After del
 <class 'pandas.io.pytables.HDFStore'>
File path: /tmp/tmp83t_wzzg

Before close True
After close False
            0         1         2         3
364  0.753342  0.381158  1.289753  0.673181


In [9]:
!pip install openpyxl

    100% |████████████████████████████████| 174kB 3.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for openpyxl ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/a7/88/96/29c1f91ba5a9b94dfc39a9f6f72d0eb92d6f0d917cf2341a3f
  Running setup.py bdist_wheel for jdcal ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/0f/63/92/19ac65ed64189de4d662f269d39dd08a887258842ad2f29549
  Running setup.py bdist_wheel for et-xmlfile ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/99/f6/53/5e18f3ff4ce36c990fa90ebdf2b80cd9b44dc461f750a1a77c
Successfully built openpyxl jdcal et-xmlfile


# Reading and writing to Excel

In [12]:
!pip install xlrd

  Using cached xlrd-1.1.0-py2.py3-none-any.whl


In [13]:
import numpy as np
import pandas as pd
from tempfile import NamedTemporaryFile

np.random.seed(42)
a = np.random.randn(365, 4)

tmpf = NamedTemporaryFile(suffix='.xlsx')
df = pd.DataFrame(a)
print(tmpf.name)
df.to_excel(tmpf.name, sheet_name='Random Data')
print("Means\n", pd.read_excel(tmpf.name, 'Random Data').mean())

/tmp/tmpvlt7xmwp.xlsx
Means
 0    0.037860
1    0.024483
2    0.059836
3    0.058417
dtype: float64


# Using REST and JSON

In [14]:
import json

json_str = '{"country":"Netherlands","dma_code":"0","timezone":"Europe\/Amsterdam","area_code":"0","ip":"46.19.37.108","asn":"AS196752","continent_code":"EU","isp":"Tilaa V.O.F.","longitude":5.75,"latitude":52.5,"country_code":"NL","country_code3":"NLD"}'

data = json.loads(json_str)
print("Country", data["country"])
data["country"] = "Brazil"
print(json.dumps(data))

Country Netherlands
{"country": "Brazil", "dma_code": "0", "timezone": "Europe/Amsterdam", "area_code": "0", "ip": "46.19.37.108", "asn": "AS196752", "continent_code": "EU", "isp": "Tilaa V.O.F.", "longitude": 5.75, "latitude": 52.5, "country_code": "NL", "country_code3": "NLD"}


<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: Dep

In [15]:
import pandas as pd

json_str = '{"country":"Netherlands","dma_code":"0","timezone":"Europe\/Amsterdam","area_code":"0","ip":"46.19.37.108","asn":"AS196752","continent_code":"EU","isp":"Tilaa V.O.F.","longitude":5.75,"latitude":52.5,"country_code":"NL","country_code3":"NLD"}'

data = pd.read_json(json_str, typ='series')
print("Series\n", data)

data["country"] = "Brazil"
print("New Series\n", data.to_json())

Series
 area_code                        0
asn                       AS196752
continent_code                  EU
country                Netherlands
country_code                    NL
country_code3                  NLD
dma_code                         0
ip                    46.19.37.108
isp                   Tilaa V.O.F.
latitude                      52.5
longitude                     5.75
timezone          Europe/Amsterdam
dtype: object
New Series
 {"area_code":"0","asn":"AS196752","continent_code":"EU","country":"Brazil","country_code":"NL","country_code3":"NLD","dma_code":"0","ip":"46.19.37.108","isp":"Tilaa V.O.F.","latitude":52.5,"longitude":5.75,"timezone":"Europe\/Amsterdam"}


<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: DeprecationWarning: invalid escape sequence \/
<input>:3: Dep

# Parsing RSS and Atom Feeds

In [17]:
!pip install feedparser

    100% |████████████████████████████████| 1.2MB 1.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for feedparser ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/15/ce/10/b500f745822ea6db6ea8ed225c06b15c000d71016b89ef9037
Successfully built feedparser


In [20]:
import feedparser as fp

rss = fp.parse("http://www.packtpub.com/rss.xml")

print("# Entries", len(rss.entries))

# Entries 10


In [21]:
for i, entry in enumerate(rss.entries):
   if "Java" in entry.summary:
      print(i, entry.title)
      print(entry.summary)

# Parsing HTML with Beautiful Soup

In [23]:
!pip install bs4

    100% |████████████████████████████████| 92kB 2.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/84/67/d4/9e09d9d5adede2ee1c7b7e8775ba3fbb04d07c4f946f0e4f11
Successfully built bs4


In [24]:
from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(open('loremIpsum.html'),"lxml")

print("First div\n", soup.div)
print("First div class", soup.div['class'])

print("First dfn text", soup.dl.dt.dfn.text)

for link in soup.find_all('a'):
   print("Link text", link.string, "URL", link.get('href'))

# Omitting find_all
for i, div in enumerate(soup('div')):
   print(i, div.contents)


#Div with id=official
official_div = soup.find_all("div", id="official")
print("Official Version", official_div[0].contents[2].strip())

print("# elements with class", len(soup.find_all(class_=True)))

tile_class = soup.find_all("div", class_="tile")
print("# Tile classes", len(tile_class))

print("# Divs with class containing tile", len(soup.find_all("div", class_=re.compile("tile"))))

print("Using CSS selector\n", soup.select('div.notile'))
print("Selecting ordered list list items\n", soup.select("ol > li")[:2])
print("Second list item in ordered list", soup.select("ol > li:nth-of-type(2)"))

print("Searching for text string", soup.find_all(text=re.compile("2014")))

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?